In [1]:
import sys
import cartopy
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import Polygon
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker
from scipy.interpolate import griddata
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature

sys.path.insert(1,'/g/data/w97/mm3972/scripts/Drought/drght_2017-2019')
from common_utils import *

In [2]:
LAI_obs_path   = "/g/data/w97/mm3972/data/MODIS/whittakerSmoothed_MCD15A3H_c61_LAI_for_WRF_daily_20170101_20200630.nc"
LAI_wrf_path   = "/g/data/w97/mm3972/model/wrf/NUWRF/LISWRF_configs/drght_2017_2019_bl_pbl2_mp4_sf_sfclay2/LIS_output/"
wrf_path       = "/g/data/w97/mm3972/model/wrf/NUWRF/LISWRF_configs/drght_2017_2019_bl_pbl2_mp4_sf_sfclay2/WRF_output/wrfout_d01_2017-01-01_11:00:00"

time_s = datetime(2017,1,1,0,0,0,0)
time_e = datetime(2020,6,30,23,59,0,0)

In [3]:
# read WRF LAI
LAI_wrf = []

for yr in np.arange(2017,2020):
    for mth in np.arange(1,13):
        if mth < 10:
            LAI_wrf_file = LAI_wrf_path + "LIS.CABLE."+str(yr)+"0"+str(mth)+"-"+str(yr)+"0"+str(mth)+".d01.nc"
        else:
            LAI_wrf_file = LAI_wrf_path + "LIS.CABLE."+str(yr)+str(mth)+"-"+str(yr)+str(mth)+".d01.nc"

        time_wrf, LAI_wrf_tmp = read_var(LAI_wrf_file, "LAI_inst") 
        print(len(np.shape(LAI_wrf_tmp)))

        LAI_wrf.append(time_clip_to_day(time_wrf, LAI_wrf, time_s, time_e),axis=0)
   

LAI_inst
3


UnboundLocalError: local variable 'var_tmp' referenced before assignment

In [ ]:
# read obs LAI
time_obs,LAI_obs_tmp = read_var(LAI_obs_path, "LAI") 
LAI_obs              = time_clip_to_day(time_obs, LAI_obs_tmp, time_s, time_e)

# read lat and lon outs
wrf          = Dataset(wrf_path,  mode='r')
lons         = wrf.variables['XLONG'][0,:,:]
lats         = wrf.variables['XLAT'][0,:,:]

LAI_obs_mean = np.nanmean(LAI_obs,axis=0)
LAI_wrf_mean = np.nanmean(LAI_wrf,axis=0)

LAI_obs_time_series = np.nanmean(LAI_obs,axis=(1,2))
LAI_wrf_time_series = np.nanmean(LAI_wrf,axis=(1,2))



In [ ]:


# ================== Start Plotting =================
fig = plt.figure(figsize=(6,5))
ax = plt.axes(projection=ccrs.PlateCarree())

plt.rcParams['text.usetex']     = False
plt.rcParams['font.family']     = "sans-serif"
plt.rcParams['font.serif']      = "Helvetica"
plt.rcParams['axes.linewidth']  = 1.5
plt.rcParams['axes.labelsize']  = 14
plt.rcParams['font.size']       = 14
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

almost_black                    = '#262626'
# change the tick colors also to the almost black
plt.rcParams['ytick.color']     = almost_black
plt.rcParams['xtick.color']     = almost_black

# change the text colors also to the almost black
plt.rcParams['text.color']      = almost_black

# Change the default axis colors from black to a slightly lighter black,
# and a little thinner (0.5 instead of 1)
plt.rcParams['axes.edgecolor']  = almost_black
plt.rcParams['axes.labelcolor'] = almost_black

# set the box type of sequence number
props = dict(boxstyle="round", facecolor='white', alpha=0.0, ec='white')

# =============== CHANGE HERE ===============
clevs = [-0.4,-0.3,-0.2,-0.1,-0.05,-0.01,0.01,0.05,0.1,0.2,0.3,0.4]
# clevs = [0,2,4,6,8,10,12,14,16,18,20]
cmap  = plt.cm.seismic_r

# start plotting
ax.set_extent([135,155,-40,-25])
ax.coastlines(resolution="50m",linewidth=1)

# Add gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='black', linestyle='--')
gl.xlabels_top   = False
gl.ylabels_right = False
gl.xlines        = True

gl.xlocator  = mticker.FixedLocator([135,140,145,150,155])
gl.ylocator  = mticker.FixedLocator([-40,-35,-30,-25])

gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size':10, 'color':'black'}
gl.ylabel_style = {'size':10, 'color':'black'}

plt.contourf(lons, lats, wb_diff[l,:,:], clevs, transform=ccrs.PlateCarree(), cmap=cmap, extend='both') # 

cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.ax.tick_params(labelsize=10)
plt.title("wb_diff", size=16)

message = "lis_rst_dry-cli_wb_layer"+str(l)

plt.savefig('/g/data/w97/mm3972/scripts/Drought/drght_2017-2019/plots/WTD_sudden_change/spatial_map_'+message+'.png',dpi=300)


# ================== Start Plotting =================
fig = plt.figure(figsize=(6,5))
ax = plt.axes(projection=ccrs.PlateCarree())

plt.rcParams['text.usetex']     = False
plt.rcParams['font.family']     = "sans-serif"
plt.rcParams['font.serif']      = "Helvetica"
plt.rcParams['axes.linewidth']  = 1.5
plt.rcParams['axes.labelsize']  = 14
plt.rcParams['font.size']       = 14
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

almost_black                    = '#262626'
# change the tick colors also to the almost black
plt.rcParams['ytick.color']     = almost_black
plt.rcParams['xtick.color']     = almost_black

# change the text colors also to the almost black
plt.rcParams['text.color']      = almost_black

# Change the default axis colors from black to a slightly lighter black,
# and a little thinner (0.5 instead of 1)
plt.rcParams['axes.edgecolor']  = almost_black
plt.rcParams['axes.labelcolor'] = almost_black

# set the box type of sequence number
props = dict(boxstyle="round", facecolor='white', alpha=0.0, ec='white')

# =============== CHANGE HERE ===============
clevs = [-0.4,-0.3,-0.2,-0.1,-0.05,0.05,0.1,0.2,0.3,0.4]
# clevs = [0,2,4,6,8,10,12,14,16,18,20]
cmap  = plt.cm.seismic

# start plotting
ax.set_extent([135,155,-40,-25])
ax.coastlines(resolution="50m",linewidth=1)

# Add gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='black', linestyle='--')
gl.xlabels_top   = False
gl.ylabels_right = False
gl.xlines        = True

gl.xlocator  = mticker.FixedLocator([135,140,145,150,155])
gl.ylocator  = mticker.FixedLocator([-40,-35,-30,-25])

gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size':10, 'color':'black'}
gl.ylabel_style = {'size':10, 'color':'black'}

plt.contourf(lons, lats, GWwb_diff, clevs, transform=ccrs.PlateCarree(), cmap=cmap, extend='both') # 

cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.ax.tick_params(labelsize=10)
plt.title("WTD_diff", size=16)

message = "lis_rst_dry-cli_gwwb"

plt.savefig('/g/data/w97/mm3972/scripts/Drought/drght_2017-2019/plots/WTD_sudden_change/spatial_map_'+message+'.png',dpi=300)     